# MultiBody

In this notebook we will see what compose a MultiBody and how to create it.

A MultiBody is a list of rigid bodies linked together by joints. We use it to represent a robot.
The [RBDyn](https://github.com/jorisv/RBDyn) library support kinematic tree and kinematic chain, but don't support closed loop kinematic structure. This last structure is addressed at a higher level in the [Tasks](https://github.com/jorisv/Tasks) library.

In this tutorial we will see first the [Body](#Body) and [Joint](#Joint) class that represent respectively link inertia and articulation type.

Then the [MultiBodyGraph](#MultiBodyGraph) class will be quickly introduced. This important class allow to store an abstract representation of the robot. We will see this class in depth in a next tutorial.

Finally we will show the [MultiBody](#MultiBody) and the [MultiBodyConfig](#MultiBodyConfig) class. The first one is a compact representation of a robot, while the second hold state value of the robot and algorithm results (articulated configuration, body configuration, external force applied on each body...)

In [32]:
%gui wx
from IPython.display import Image
from tvtk.tools import ivtk
from graph import Axis

# create the viewer
viewer = ivtk.viewer()
viewer.size = (640, 480)

# TODO make camera work...
# viewer.scene.camera.focal_point = (0.193, 0.447, 0.)
# viewer.scene.camera.position = (0.193, 0.447, 2.742)

display_number = 1
def display():
    global display_number
    path = 'img/SpaceVecAlg/%s.png' % display_number
    display_number += 1
    viewer.scene.save(path)
    return Image(path)

In [33]:
import numpy as np
import eigen3 as e
import spacevecalg as sva
import rbdyn as rbd

## Body

`rbd::Body` is a simple class that store a body name, id and inertia.

The body name must be an unique string value associated with the body.
Same for the id, that is an unique integer value associated with the body.

In python we will commonly use name to reference a body while the RBDyn API use id.

Finally the body store a `sva::RBInertia` class that will hold the body inertia value (mass, center of mass and rotational inertia at body origin).

In [34]:
b1m = 2. # 2kg
b1c = e.Vector3d(0.5, 0.5, 0.) # center of mass at [0.5, 0.5, 0.] from the body origin
b1rI = e.Matrix3d.Identity() # rotational inertia at body origin
# here the second argument is the body first moment of mass (h = m c)
b1I = sva.RBInertia(b1m, b1m*b1c, b1rI)
# first body constructor that take sva.RBInertia as firt param
b1 = rbd.Body(b1I, 1, 'b1')

b2m = 4. # 4kg
b2c = e.Vector3d(0., 0.5, 0.) # center of mass at [0., 0.5, 0.] from the body origin
b2rI = e.Matrix3d.Identity() # rotational inertia at body origin
# here the second argument is the body center of mass (not the first moment of mass)
b2 = rbd.Body(b2m, b2c, b2rI, 2, 'b2')

b3 = rbd.Body(1., e.Vector3d.Zero(), e.Matrix3d.Identity(), 3, 'b3')
b4 = rbd.Body(1., e.Vector3d.Zero(), e.Matrix3d.Identity(), 4, 'b4')
b5 = rbd.Body(1., e.Vector3d.Zero(), e.Matrix3d.Identity(), 5, 'b5')

## Joint

The `rbd::Joint` class is more complex than `rbd::Body` especially because this class old 7 kind of joint:
 * [Fixed](#Fixed) : Fixed joint
 * [Revolute](#Revolute) : Rotate around one axis
 * [Prismatic](#Prismatic) : Translate on one axis
 * [Spherical](#Spherical) : Spherical rotation
 * [Planar](#Planar) : Rotation around the normal axis and translation on the tangent plan
 * [Cylindrical](#Cylindrical) : Rotation and translation around one axis
 * [Free](#Free) : 6DoF transformation
 
In a first time we will see common Joint methods, then we will see the particularity of each joint type.
 
Like Body, each Joint should have an unique string and integer Id.
Joint also have a forward attribute. If this attribute is false the polarity of the joint is reversed, that mean transformation and motion vector return by the joint will be inversed.
Also some joint type can take an axis as parameter.

Two constructor exist:
```c++
Joint(Type type, const Eigen::Vector3d& axis, bool forward, int id, std::string name)
Joint(Type type, bool forward, int id, std::string name)
```

Each Joint have a number of parameters `int params() const` ($ P $) and a number of degree of freedom (DoF) `int dof() const` ($ A $). Those value give the size of the articulated position vector $ q_i \in \mathbb{R}^{P} $ and the articulated velocity vector $ \alpha_i \in \mathbb{R}^{A} $ of the joint $ i $.
The articulated velocity vector is for all joint expressed in the successor frame of the joint, that mean in the frame after the transformation of the joint.

The `Eigen::MatrixXd motionSubspace() const` method return the Motion Subspace matrix $ S \in \mathbb{R}^{6,A} $ that allow to transform the articulated position vector and all his time derivative into a motion vector $ \hat{v}_i = S_i \alpha_i $.

The `sva::PTransformd pose(const std::vector<double>& q) const` return the transformation of a joint associated to a articulated position vector $ q_i $.

The `sva::MotionVecd motion(const std::vector<double>& alpha) const` return the motion vector of a joint associated to a articulation velocity vector and his all his time derivative $ \alpha_i $.

Finally the `std::vector<double> zeroParam() const` and `std::vector<double> zeroDof() const` return the articulated position vector $ q_i $ and articulated velocity vector $ \alpha_i $ that produce respectively an identity transform and a null motion vector.

In [35]:
def jointResume(j):
    print 'P =', j.params()
    print 'A =', j.dof()
    print 'qZero =', list(j.zeroParam())
    print 'alphaZero =', list(j.zeroDof())
    print 'motion subspace ='
    print e.toNumpy(j.motionSubspace())

### Fixed

The fixed joint don't take any parameter and have zero degree of freedom.

In [36]:
jFix = rbd.Joint(rbd.Joint.Fixed, True, 0, 'jFix')
jointResume(jFix)

P = 0
A = 0
qZero = []
alphaZero = []
motion subspace =
[]


### Revolute

The revolute joint rotate around one axis given in parameter and have 1 parameter and 1 degree of freedom.

In [37]:
# Revolute joint around X axis
jRev = rbd.Joint(rbd.Joint.Rev, e.Vector3d.UnitX(), True, 0, 'jRev')
jointResume(jRev)
print 'rotation:'
print jRev.pose([np.pi/2.]).rotation()
print 'angular motion:', jRev.motion([1.]).angular().transpose()

P = 1
A = 1
qZero = [0.0]
alphaZero = [0.0]
motion subspace =
[[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
rotation:
          1           0           0
          0 6.12323e-17           1
          0          -1 6.12323e-17
angular motion: 1 0 0


### Prismatic

The prismatic joint translate around one axis given in parameter and have 1 parameter and 1 degree of freedom.

In [38]:
# Prismatic joint at Y axis
jPrism = rbd.Joint(rbd.Joint.Prism, e.Vector3d.UnitY(), True, 0, 'jPrism')
jointResume(jPrism)
print 'translation:', jPrism.pose([1.]).translation().transpose()
print 'linear motion:', jPrism.motion([0.1]).linear().transpose()

P = 1
A = 1
qZero = [0.0]
alphaZero = [0.0]
motion subspace =
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]]
translation: 0 1 0
linear motion:   0 0.1   0


### Spherical

The sphercial joint is a particular one. It use an unit quaternion (qw, qx, qy, qz) to encode a 3D rotation. Because of the quaternion this joint have 4 parameters [qw, qx, qy, qz] and 3 degree of freedom [wx, wy, wz].

In [39]:
jSph = rbd.Joint(rbd.Joint.Spherical, True, 0, 'jSph')
jointResume(jSph)
q = e.Quaterniond(np.pi/4., e.Vector3d(1., 0., 0.5).normalized())
qParam = [q.w(), q.x(), q.y(), q.z()]
print 'rotation:'
print jSph.pose(qParam).rotation()
print 'angular motion:', jSph.motion([1., 0.2, 0.5]).angular().transpose()

P = 4
A = 3
qZero = [1.0, 0.0, 0.0, 0.0]
alphaZero = [0.0, 0.0, 0.0]
motion subspace =
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
rotation:
 0.941421  0.316228  0.117157
-0.316228  0.707107  0.632456
 0.117157 -0.632456  0.765685
angular motion:   1 0.2 0.5


### Planar

The planar joint rotate around the normal axis (Z) and translate around the tangent plane (X,Y) AFTER the rotation.
That mean the rotation is done first, then the translation is done in the rotated frame.
This joint take 3 parameters [rotation_Z, translation_X, translation_Y] and have 3 degree of freedom in the same order.

In [40]:
jPlan = rbd.Joint(rbd.Joint.Planar, True, 0, 'jPlan')
jointResume(jPlan)
qParam = [np.pi/2., 0.2, 0.1]
print 'translation:', jPlan.pose(qParam).translation().transpose()
print 'rotation:'
print jPlan.pose(qParam).rotation()
print 'motion:', jPlan.motion([0.2, 0.5, -0.5])

P = 3
A = 3
qZero = [0.0, 0.0, 0.0]
alphaZero = [0.0, 0.0, 0.0]
motion subspace =
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
translation: -0.1 0.2   0
rotation:
6.12323e-17           1           0
         -1 6.12323e-17           0
          0           0           1
motion:    0    0  0.2  0.5 -0.5    0


### Cylindrical

The cylindrical joint translate around one axis and then rotation around the same axis.
It take 2 parameters [rotation_axis, translation_axis] and 2 degree of freedom in the same order.

In [41]:
jCyl = rbd.Joint(rbd.Joint.Cylindrical, e.Vector3d.UnitZ(), True, 0, 'jCyl')
jointResume(jCyl)
qParam = [np.pi/2., 0.4]
print 'translation:', jCyl.pose(qParam).translation().transpose()
print 'rotation:'
print jCyl.pose(qParam).rotation()
print 'motion:', jCyl.motion([0.4, -0.7])

P = 2
A = 2
qZero = [0.0, 0.0]
alphaZero = [0.0, 0.0]
motion subspace =
[[ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  1.]]
translation:   0   0 0.4
rotation:
6.12323e-17           1           0
         -1 6.12323e-17           0
          0           0           1
motion:    0    0  0.4    0    0 -0.7


### Free

The free joint is a 6Dof transformation. It make a 3d translation and a 3d rotation.
Like the spherical joint, the free joint use an unit quaternion for rotation that make this joint have don't have the same number of parameters and degree of freedom.

This joint have 7 parameters [qw, qx, qy, qz, tx, ty, tz] and 6 degree of freedom [wx, wy, wz, vx, vy, vz].

In [ ]:
jFree = rbd.Joint(rbd.Joint.Free, True, 0, 'jFree')
jointResume(jFree)
q = e.Quaterniond(-np.pi/3., e.Vector3d(0.5, 1., 0.2).normalized())
qParam = [q.w(), q.x(), q.y(), q.z(), 0.2, -0.4, 0.7]
print 'translation:', jFree.pose(qParam).translation().transpose()
print 'rotation:'
print jFree.pose(qParam).rotation()
print 'motion:', j.motion([0.2, 0.5, -0.5])

## MultiBodyGraph

## MultiBody

## MultiBodyConfig